## Laden und durchsuchen von eigenen Daten

1. Zuerst laden wir die Dokumente aus dem docs Verzeichniss
2. Dann teilen wir die Dokumente in kleine Segmente auf
3. Dann erstellen wir daraus einen Index

In [1]:
from langchain_community.document_loaders import DirectoryLoader
loader = DirectoryLoader('./docs', glob="**/*.md")
data = loader.load()
len(data)

3

In [2]:
print(f"Zeichen in Text {len(data[1].page_content)}")
data[1]

Zeichen in Text 262


Document(metadata={'source': 'docs/personal.md'}, page_content='Lohnbuchhaltung\n\nFrau Müller in Raum 123 übernimmt bei uns die Lohnbuchhaltung. Lohnbuchhaltung Name: Frau Müller Tel.: 0351 1231231 Email.: frau.mueller@telekom.de\n\n## Hausmeister Der Hausmeister ist Herr Schulz Tel.: 0351 3432423 Email.: herr.schulz@telekom.de')

### Wie können wir die Texte in kürzere Abschnitte unterteilen?

Web Demo [Chunk Visualizer](https://huggingface.co/spaces/m-ric/chunk_visualizer)

In [3]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=400, chunk_overlap=50)
all_splits = text_splitter.split_documents(data)

print(len(all_splits))
all_splits

24


[Document(metadata={'source': 'docs/mms.md'}, page_content='Telekom MMS\n\nDie Deutsche Telekom MMS GmbH (Telekom MMS) ist ein Digital-Dienstleister, der sich als Begleiter von Großkonzernen und mittelständischen Unternehmen bei der digitalen Transformation versteht.'),
 Document(metadata={'source': 'docs/mms.md'}, page_content='Über 2.200 Mitarbeitende an neun Standorten in Deutschland entwickeln zukunftsfähige Geschäftsmodelle für digitale Erlebnisse. Telekom MMS bietet ein dynamisches Web- und Application-Management und sorgt mit dem hauseigenen akkreditierten Test-Center für höchste Softwarequalität, Barrierefreiheit und IT-Sicherheit.'),
 Document(metadata={'source': 'docs/mms.md'}, page_content='Das Unternehmen zählt zum Konzern Deutsche Telekom AG und tritt ebenfalls mit dem T-Logo auf. Das Unternehmen hat Standorte in Dresden, Leipzig, Jena, Berlin, Rostock, München, Stuttgart, Bonn und Hamburg. Geschichte'),
 Document(metadata={'source': 'docs/mms.md'}, page_content='Das Unter

In [4]:
import chromadb
from chromadb.utils import embedding_functions

# setup Chroma in-memory, for easy prototyping. Can add persistence easily!
chroma = chromadb.Client()

# Create collection. get_collection, get_or_create_collection, delete_collection also available!
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="paraphrase-multilingual-MiniLM-L12-v2")
try:
    chroma.delete_collection("documents")
except:
    pass

collection = chroma.get_or_create_collection("documents",embedding_function=sentence_transformer_ef)

# Add docs to the collection. Can also update and delete. Row-based API coming soon!
collection.add(
    documents=[item.page_content for item in all_splits], # we handle tokenization, embedding, and indexing automatically. You can skip that and add your own embeddings as well
    metadatas=[item.metadata for item in all_splits], # filter on these!
    ids=[str(id) for id in range(0,len(all_splits))], # unique for each doc
)

In [10]:
# Query/search 2 most similar results. You can also .get by id
results = collection.query(
    query_texts=["Wer sind die ehemaligen Geschäftsführer"],
    n_results=4,
    # where={"metadata_field": "is_equal_to_this"}, # optional filter
    #where_document={"$contains":"Oliver"}  # optional filter
)

results

{'ids': [['8', '23', '22', '4']],
 'embeddings': None,
 'documents': [['Die Geschäftsführung besteht aus einer Doppelspitze: Ralf Pechmann, mit Hauptverantwortung im Bereich Vertrieb, Marketing, Kunden und Marcus Gaube als kaufmännischer Leiter des Unternehmens. Ehemalige Geschäftsführer sind: Friedhelm Theis (1995–1998) Joachim Niemeier (1995–2005) Klaus Radermacher (1999–2002 und 2005–2007) Jens Nebendahl (2007–2013) Helmut Binder (2007–2008) Rolf Werner',
   'Personen\n\nOliver Guhr, Schulungsleiter',
   'Lohnbuchhaltung\n\nFrau Müller in Raum 123 übernimmt bei uns die Lohnbuchhaltung. Lohnbuchhaltung Name: Frau Müller Tel.: 0351 1231231 Email.: frau.mueller@telekom.de\n\n## Hausmeister Der Hausmeister ist Herr Schulz Tel.: 0351 3432423 Email.: herr.schulz@telekom.de',
   '2002 erfolgte die Umfirmierung in „T-Systems Multimedia Solutions GmbH“ als hundertprozentige Tochter der T-Systems International GmbH, die 2000 gegründet wurde und damals die bisherigen Tochtergesellschaften der 

## Aufgaben 

* Trage informationen ein - Findet sie die suche wieder?
* Wie groß sollte die Chuck Size sein?
* Finde die ehemaligen Geschäftsführer


Optional 
* Open AI Embeddings
* BGE Embeddings